In [1]:
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('../src')
from textwrap import dedent
import pandas as pd 

from llm_utils import chat_completion_openai
from configs import llm_scenarios_config, models_config


In [2]:

async def get_tags(image_description):
    prompt_template = dedent("""
    You will be analyzing a short image description and providing tags for specific categories. Here is the image description:

    <image_description>
    $IMAGE_DESCRIPTION
    </image_description>

    Based on this description, you need to provide tags for the following categories:

    1. Eroticism Level: Choose one of [Explicit, Softcore, Casual]
    - Explicit: naked intimate parts visible
    - Softcore: light erotics, suggestive but not explicit
    - Casual: non-erotic, everyday scenes

    2. Body Part Emphasis: Choose one or more of [face, breasts, pussy, legs, hands]
    - Select the body part(s) that seem to be the focus of the image

    3. Position: Choose one of [standing, lying, sitting, squatting, legs spread]
    - Select the action that best describes the person's position in the image
    
    4. Location: Choose one of [outdoors, indoors]
    
    4. Clothes: Choose one of [anything stated in description]
    
    If category is unspecified, do not include it in output.
    
    
    Analyze the image description carefully and output your final tags in the JSON format. Use lowercase for all letters.
    
    
    Response format instructions:
    1. Output ONLY valid python dict and nothing else. Start your answer with { and finish with }
    2. Response example: {'eroticism level': 'explicit', 'body part emphasis': 'pussy', 'position': 'legs spread'}

    Remember to base your analysis solely on the information provided in the image description. Do not make assumptions beyond what is explicitly stated or clearly implied by the description.
    """).strip()
    
    prompt_template = prompt_template.replace("$IMAGE_DESCRIPTION", image_description)
    messages = [{"role": "user", "content": str(prompt_template)}]

    resp = await chat_completion_openai(messages, llm_scenarios_config['tagging'])
    
    return resp

resp = await get_tags('nswf, naked, legs spread, in bathroom')
print(resp)

_client.py          :1773 2024-08-07 09:29:37,003 HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


{'answer_text': '{\n  "eroticism level": "explicit",\n  "body part emphasis": "pussy, legs",\n  "position": "legs spread",\n  "location": "indoors",\n  "clothes": "naked"\n}', 'input_tokens': 335, 'output_tokens': 50, 'execution_time': 6.1017961502075195, 'status': 'success'}


In [5]:
import json
json.loads(resp['answer_text'])

{'eroticism level': 'explicit',
 'body part emphasis': 'pussy, legs',
 'position': 'legs spread',
 'location': 'indoors',
 'clothes': 'naked'}